In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from config import llm
from transcribe_tool import youtube_transcriber

memory = MemorySaver()

tool_box = [youtube_transcriber]

agent_executor = create_react_agent(llm, tool_box)

In [ ]:
from langchain.schema import HumanMessage, AIMessage
import asyncio

async def async_input(prompt: str = "") -> str:
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(None, input, prompt)

# define main chat function
async def main():

    # conversation history
    messages = []

    print("== YOUTUBE TRANSCRIPT AGENT ==")
    print("type exit to end the convo")
    print("Aslk me anything about Youtube Vidoes")

    while True:

        # get user input
        user_input = (await async_input("\nYou: ")).strip()
        messages.append(HumanMessage(content=user_input))

        # printing to user
        print(f"\nYou: {user_input}")

        # exiting the chatbot
        if user_input.lower() in ['exit']:
            print("\nGOODBOYE!")
            break

        # continuing if the user_input does not exist
        if not user_input:
            continue

        # invoking the agent asynchronously
        response = await agent_executor.ainvoke(    
                    {"messages": messages},
                    config={"configurable": {"thread_id": "session-1"}}
                )
        
        output = response["messages"][-1].content
        messages.append(AIMessage(content=output))
        
        # printing the outpt
        print(f"\nChatbot: {output}")


In [ ]:
# main
await main()